In [2]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from torch.utils.data import DataLoader

import imageio.v3 as iio
from matplotlib.colors import Normalize

from lpfm.data.phys_dataset import SuperDataset
from lpfm.data.ind_datasets import EulerDataset

In [ ]:
# Load all well datasets
from lpfm.data.dataset_utils import get_datasets

datasets = [
    "cooled_object_pipe_flow_air",
    "heated_object_pipe_flow_air",
    "object_sym_flow_air",
    "object_sym_flow_water",
    "object_periodic_flow_water",
    "cylinder_sym_flow_water",
    "cylinder_pipe_flow_water",
    "euler_multi_quadrants_periodicBC",
    "turbulent_radiative_layer_2D",
    "rayleigh_benard",
    "shear_flow",
]

for dataset in datasets:
    data_config = {
        "datasets": [dataset],
        "data_dir": "/hpcwork/rwth1802/coding/Large-Physics-Foundation-Model/data/datasets",
        "n_steps_input": 16,
        "n_steps_output": 1,
        "dt_stride": [1,8],
        "out_shape": [256, 128],
        "max_samples_per_ds": None,
        "full_trajectory_mode": False,
    }

    loaded_datasets: dict = get_datasets(data_config, split="train")

    ds = loaded_datasets[dataset]
    print(f"{dataset} has {len(ds)} samples")
    



In [4]:
data_config = {
    "datasets": ["euler_multi_quadrants_periodicBC"],
    "data_dir": "/hpcwork/rwth1802/coding/Large-Physics-Foundation-Model/data/datasets",
    "n_steps_input": 16,
    "n_steps_output": 1,
    "dt_stride": [1,8],
    "out_shape": [256, 128],
    "max_samples_per_ds": None,
    "full_trajectory_mode": False,
}

loaded_datasets = get_datasets(data_config, split="train")
dataset = SuperDataset(loaded_datasets, (256, 128))

x, y = dataset[0]

# # take first batch
# x = x[0,...]
# y = y[0,...]

pressure = x[..., 0]
density = x[..., 1]
temperature = x[..., 2]
vel_x = x[..., 3]
vel_y = x[..., 4]

In [ ]:
time_idx = [0,1,2,3]
# Create figure with subplots
fig, axes = plt.subplots(5, len(time_idx), figsize=(15, 20))

# Plot all five physical fields for each timestep
field_names = ['Pressure', 'Density', 'Temperature', 'Velocity X', 'Velocity Y']
field_data = [pressure, density, temperature, vel_x, vel_y]
cmaps = ['plasma', 'viridis', 'inferno', 'cividis', 'magma']

for row, (field, name, cmap) in enumerate(zip(field_data, field_names, cmaps)):
    for col, t in enumerate(time_idx):
        im = axes[row, col].imshow(field[t,...], cmap=cmap)
        plt.colorbar(im, ax=axes[row, col], label=name)
        
        # Add labels and title
        axes[row, col].set_xlabel('X')
        axes[row, col].set_ylabel('Y')
        axes[row, col].set_title(f'{name} Field (t={time_idx[col]})')

plt.tight_layout()
plt.show()


In [ ]:
traj_idx = 0
vel_mag = np.linalg.norm(vel[traj_idx, :, :, :, :], axis=-1)

# reduce dim with size 1 to 0
vel_mag = np.squeeze(vel_mag)

# transpose x and y
vel_mag = np.transpose(vel_mag, (0,  2, 1))

# Create normalized colormap for consistent color scaling
norm = Normalize(vmin=vel_mag.min(), vmax=vel_mag.max())

# Create frames
frames = []
for t in range(vel_mag.shape[0]):
    # Create figure and plot
    fig, ax = plt.subplots()
    im = ax.imshow(vel_mag[t,:,:])#, norm=norm)
    plt.colorbar(im)
    ax.set_title(f'Time step {t}')
    
    # Convert plot to image array
    fig.canvas.draw()
    frame = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
    frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (4,))[:,:,:3]
    frames.append(frame)
    plt.close()

# Save as GIF
output_path = data_path.parent
output_path = output_path / (str(data_path.stem) + "_vel.gif")
print(f"Writing {len(frames)} frames to {output_path}")
iio.imwrite(output_path, frames, fps=30)
print(f"Animation saved to {output_path}")